In [21]:
target_refund_file_path='D://work//Programming//data//race_refund_datas//2024-04_2024-05.csv'
target_predict_file_path='D://work//Programming//predict//LGBMRanker01//result.csv'

import pandas as pd
import sys
sys.path.append('../')
import common
from common import const,utils
import numpy as np
import calc_methods as calc

refund = pd.read_csv(target_refund_file_path)
predict = pd.read_csv(target_predict_file_path)


In [22]:
predict

,レースID,馬番,馬のID,予想スコア,着順関連度
0,202403010101,2,2021103480,0.426926,30
1,202403010101,11,2021101732,0.314995,0
2,202403010101,7,2021100005,0.311884,28
3,202403010101,10,2021100052,0.301125,24
4,202403010101,12,2021107356,0.244796,26
...,...,...,...,...,...
7744,202409020812,10,2017100447,-2.054375,0
7745,202409020812,11,2019103543,-2.790954,22
7746,202409020812,9,2019104448,-3.877595,0
7747,202409020812,4,2018105738,-3.877595,0


In [23]:
# 予想スコアが最も高い馬を単勝で購入した場合の回収率を計算

# 出力結果
#   購入金額:55200
#   返還金額:41400.0
#   回収率:0.75

purchase_amaount = 0
return_amount = 0
purchase_count = 0
hit_count = 0

for race_id,race_info in predict.groupby('レースID'):
    target_horse = race_info.sort_values(by='予想スコア',ascending=False)[0:1]
    _return = calc.calculate_return_tansho(refund,race_id,100,target_horse['馬番'].values[0])
    if _return != 0:
        return_amount += _return
        hit_count += 1
    purchase_amaount += 100
    purchase_count+=1

print('購入金額:{}'.format(purchase_amaount))
print('返還金額:{}'.format(return_amount))
print('的中率{}'.format(hit_count/purchase_count))
print('回収率:{}'.format(return_amount/purchase_amaount))

購入金額:55200
返還金額:41400.0
的中率0.3079710144927536
回収率:0.75


In [25]:
# 閾値以上の場合のみ単勝で購入した場合の回収率を計算

# 出力結果
#   購入金額:42300
#   返還金額:32800.0
#   回収率:0.7754137115839244


# 閾値を予想スコアが0以上の場合の平均で設定
threshold = predict[predict['予想スコア']>0]['予想スコア'].mean()

purchase_amaount = 0
return_amount = 0
purchase_count = 0
hit_count = 0

for race_id,race_info in predict.groupby('レースID'):

    target_horse = race_info.sort_values(by='予想スコア',ascending=False)[0:1]
    if target_horse['予想スコア'].values[0] < threshold:
        continue
    _return = calc.calculate_return_tansho(refund,race_id,100,target_horse['馬番'].values[0])
    if _return != 0:
        return_amount += _return
        hit_count += 1
    purchase_amaount += 100
    purchase_count+=1

print('購入金額:{}'.format(purchase_amaount))
print('返還金額:{}'.format(return_amount))
print('的中率{}'.format(hit_count/purchase_count))
print('回収率:{}'.format(return_amount/purchase_amaount))

購入金額:42300
返還金額:32800.0
的中率0.35224586288416077
回収率:0.7754137115839244


In [26]:
# 予想スコアが最も高い馬を複勝で購入した場合の回収率を計算

# 出力結果
#   購入金額:55200
#   返還金額:47130.0
#   回収率:0.8538043478260869

purchase_amaount = 0
return_amount = 0
purchase_count = 0
hit_count = 0

for race_id,race_info in predict.groupby('レースID'):
    target_horse = race_info.sort_values(by='予想スコア',ascending=False)[0:1]
    _return = calc.calculate_return_hukusho(refund,race_id,100,target_horse['馬番'].values[0])
    if _return != 0:
        return_amount += _return
        hit_count += 1
    purchase_amaount += 100
    purchase_count+=1


print('購入金額:{}'.format(purchase_amaount))
print('返還金額:{}'.format(return_amount))
print('的中率{}'.format(hit_count/purchase_count))
print('回収率:{}'.format(return_amount/purchase_amaount))

購入金額:55200
返還金額:47130.0
的中率0.6503623188405797
回収率:0.8538043478260869


In [27]:
# 予想スコアが上位の馬2頭をワイドで購入した場合の回収率を計算

# 出力結果
#   購入金額:55200
#   返還金額:29950.0
#   回収率:0.542572463768116

purchase_amaount = 0
return_amount = 0
purchase_count = 0
hit_count = 0

for race_id,race_info in predict.groupby('レースID'):
    target_horses = race_info.sort_values(by='予想スコア',ascending=False)[0:2]
    _return = calc.calculate_return_wide(refund,race_id,100,target_horses['馬番'].values)
    if _return != 0:
        return_amount += _return
        hit_count += 1
    purchase_amaount += 100
    purchase_count+=1


print('購入金額:{}'.format(purchase_amaount))
print('返還金額:{}'.format(return_amount))
print('的中率{}'.format(hit_count/purchase_count))
print('回収率:{}'.format(return_amount/purchase_amaount))

購入金額:55200
返還金額:46550.0
的中率0.3242753623188406
回収率:0.8432971014492754


In [28]:
# 予想スコアが上位の半分を3連複ボックスで購入した場合の回収率を計算

# 出力結果
#   購入金額:31300
#   返還金額:30910.0
#   回収率:0.9875399361022364

from itertools import combinations  

purchase_amaount = 0
return_amount = 0
purchase_count = 0
hit_count = 0


for race_id,race_info in predict.groupby('レースID'):
    # 5分の1を購入
    target_num=int(len(race_info)/5)
    target_horses = race_info.sort_values(by='予想スコア',ascending=False)[0:target_num]
    combinations_list = list(combinations(target_horses['馬番'].values, 3))
    for combination in combinations_list:
        _return = calc.calculate_return_3renpuku(refund,race_id,100,list(combination)) 
        if _return != 0:
            return_amount += _return
            hit_count += 1
        purchase_amaount += 100
        purchase_count+=1


print('購入金額:{}'.format(purchase_amaount))
print('返還金額:{}'.format(return_amount))
print('的中率{}'.format(hit_count/purchase_count))
print('回収率:{}'.format(return_amount/purchase_amaount))

購入金額:31300
返還金額:30910.0
的中率0.08626198083067092
回収率:0.9875399361022364


In [29]:
# 予想スコアが上位の半分を3連単ボックスで購入した場合の回収率を計算

# 出力結果
#   購入金額:187800
#   返還金額:173560.0
#   回収率:0.924174653887114

from itertools import permutations  

purchase_amaount = 0
return_amount = 0
purchase_count = 0
hit_count = 0

for race_id,race_info in predict.groupby('レースID'):
    target_num=int(len(race_info)/5)
    target_horses = race_info.sort_values(by='予想スコア',ascending=False)[0:target_num]
    combinations_list = list(permutations(target_horses['馬番'].values, 3))
    for combination in combinations_list:
        _return = calc.calculate_return_3rentan(refund,race_id,100,list(combination)) 
        if _return != 0:
            return_amount += _return
            hit_count += 1
        purchase_amaount += 100
        purchase_count+=1

print('購入金額:{}'.format(purchase_amaount))
print('返還金額:{}'.format(return_amount))
print('的中率{}'.format(hit_count/purchase_count))
print('回収率:{}'.format(return_amount/purchase_amaount))

購入金額:187800
返還金額:173560.0
的中率0.01437699680511182
回収率:0.924174653887114
